# Advanced Expression Manipulation

## Understanding the Expression Tree

In [1]:
from sympy import *
x, y, z = symbols('x y z')

In [2]:
expr = x**2 + x*y
srepr(expr)

"Add(Pow(Symbol('x'), Integer(2)), Mul(Symbol('x'), Symbol('y')))"

The easiest way to tear this apart is to look at a diagram of the expression tree:

![expression tree](./img/2023-12-06-11-09-44.png)

In [3]:
x = symbols('x')

When we write x**2, this creates a Pow object. Pow is short for “power”.

In [4]:
srepr(x**2)

"Pow(Symbol('x'), Integer(2))"

Note that in the srepr output, we see Integer(2), the SymPy version of integers, even though technically, we input 2, a Python int. In general, whenever you combine a SymPy object with a non-SymPy object via some function or operation, the non-SymPy object will be converted into a SymPy object. The function that does this is sympify.

In [6]:
type(2)

int

In [7]:
type(sympify(2))

sympy.core.numbers.Integer

We have seen that x**2 is represented as Pow(x, 2). What about x*y? As we might expect, this is the multiplication of x and y. The SymPy class for multiplication is Mul.

In [8]:
srepr(x*y)

"Mul(Symbol('x'), Symbol('y'))"

SymPy expression trees can have many branches, and can be quite deep or quite broad. Here is a more complicated example

In [9]:
expr = sin(x*y)/2 - x**2 + 1/y
srepr(expr)

"Add(Mul(Integer(-1), Pow(Symbol('x'), Integer(2))), Mul(Rational(1, 2), sin(Mul(Symbol('x'), Symbol('y')))), Pow(Symbol('y'), Integer(-1)))"

Here is diagram

![expression tree2](./img/2023-12-06-11-30-18.png)

## Recursing through an Expression Tree

### func

In [10]:
expr = Add(x, x)
expr.func

sympy.core.mul.Mul

In [11]:
expr

2*x

Add(x, x), i.e., x + x, was automatically converted into Mul(2, x), i.e., 2*x, which is a Mul. SymPy classes make heavy use of the `__new__` class constructor, which, unlike `__init__`, allows a different class to be returned from the constructor.

In [12]:
Integer(2).func

sympy.core.numbers.Integer

In [13]:
Integer(0).func

sympy.core.numbers.Zero

In [14]:
Integer(-1).func

sympy.core.numbers.NegativeOne

For the most part, these issues will not bother us. The special classes Zero, One, NegativeOne, and so on are subclasses of Integer, so as long as you use isinstance, it will not be an issue.

### args

In [16]:
expr = 3*y**2*x
expr.func

sympy.core.mul.Mul

In [17]:
expr.args

(3, x, y**2)

In [18]:
expr = y**2*3*x
expr.args

(3, x, y**2)

Mul’s args are sorted, so that the same Mul will have the same args. But the sorting is based on some criteria designed to make the sorting unique and efficient that has no mathematical significance.

In [19]:
expr.args[2]

y**2

In [20]:
expr.args[2].args

(y, 2)

In [21]:
y.args

()

In [22]:
Integer(2).args

()

They both have empty args. In SymPy, empty args signal that we have hit a leaf of the expression tree.

Key Invariant:
- Every well-formed SymPy expression must either have empty args or satisfy $ expr == expr.func(*expr.args) $



### Walking the Tree

In [23]:
def pre(expr):
    print(expr)
    for arg in expr.args:
        pre(arg)

In [24]:
expr = x*y + 1
pre(expr)

x*y + 1
1
x*y
x
y


Such traversals are so common in SymPy that the generator functions `preorder_traversal` and `postorder_traversal` are provided to make such traversals easy. We could have also written our algorithm as

In [26]:
for arg in postorder_traversal(expr):
    print(arg)

1
x
y
x*y
x*y + 1


## Prevent expression evaluation

In [27]:
from sympy import Add
from sympy.abc import x, y, z
x + x

x + x

In [28]:
Add(x, x)

2*x

In [29]:
Add(x, x, evaluate=False)

x + x

Note that evaluate=False won’t prevent future evaluation in later usages of the expression:

In [30]:
expr = Add(x, x, evaluate=False)
expr
expr + x

3*x

That’s why the class UnevaluatedExpr comes handy. UnevaluatedExpr is a method provided by SymPy which lets the user keep an expression unevaluated. By unevaluated it is meant that the value inside of it will not interact with the expressions outside of it to give simplified outputs. For example:

In [32]:
from sympy import UnevaluatedExpr
expr = x + UnevaluatedExpr(x)
expr

x + x

In [33]:
x + expr

2*x + x

In [35]:
from sympy import *
from sympy.abc import x, y, z
uexpr = UnevaluatedExpr(S.One*5/7)*UnevaluatedExpr(S.One*3/4)
uexpr

(5/7)*(3/4)

In [36]:
x*UnevaluatedExpr(1/x)

x*1/x

A point to be noted is that UnevaluatedExpr cannot prevent the evaluation of an expression which is given as argument. For example:

In [38]:
expr1 = UnevaluatedExpr(x + x)
expr1

2*x

In [39]:
expr2 = sympify('x + x', evaluate=False)
expr2

x + x

Remember that expr2 will be evaluated if included into another expression. Combine both of the methods to prevent both inside and outside evaluations:

In [40]:
UnevaluatedExpr(sympify("x + x", evaluate=False)) + y

y + (x + x)

UnevaluatedExpr is supported by SymPy printers and can be used to print the result in different output forms. For example

In [41]:
from sympy import latex
uexpr = UnevaluatedExpr(S.One*5/7)*UnevaluatedExpr(S.One*3/4)
print(latex(uexpr))

\frac{5}{7} \cdot \frac{3}{4}


In [42]:
print(latex(uexpr.doit()))

\frac{15}{28}
